# Passivation of Silicon (001) Surface

## 0. Introduction.

In this notebook, we will demonstrate how to passivate a reconstructed silicon (100) surface with hydrogen atoms. Following the publication: 

> **Hansen, U., & Vogl, P.**
> "Hydrogen passivation of silicon surfaces: A classical molecular-dynamics study."
> Physical Review B, 57(20), 13295–13304. (1998).
> [DOI: 10.1103/PhysRevB.57.13295](https://doi.org/10.1103/PhysRevB.57.13295)


This notebook will focus on recreating the material from Fig. 8.:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/passivation/passivation_surface_silicon/0-figure-from-manuscript.webp" alt="H-Passivated Silicon (100)" width="400"/>

## 1. Prepare the environment
### 1.1. Set up the notebook

In [ ]:
# Material selection
MATERIAL_NAME = "Silicon.*100.*reconstructed"  # Name (regex) of the material to import from Standata

# Passivation parameters
PASSIVANT = "H"  # Chemical symbol of passivating atom
BOND_LENGTH = 1.46  # Distance from surface to passivant, in Angstroms
SURFACE = "top"  # Which surface to passivate: "top", "bottom" or "both"

# Surface detection parameters
SHADOWING_RADIUS = 1.8  # Radius to exclude subsurface atoms, in Angstroms
DEPTH = 0.5  # How deep to look for surface atoms, in Angstroms

# Slab parameters for creating a new slab if provided material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 3,
    "vacuum": 10.0,
    "use_orthogonal_z": True,
    "xy_supercell_matrix": [[3, 0], [0, 3]]
}

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [1, 1, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Load and preview input material

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

slab = Material(Materials.get_by_name_first_match(MATERIAL_NAME))

## 2. Prepare the material
### 2.1. Visualize original material

In [ ]:
from utils.visualize import visualize_materials

if SHOW_INTERMEDIATE_STEPS:
    print("Initial slab structure:")
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

### 2.2. Adjust structure to match the one from the manuscript

In [ ]:
# Find coordinates of the Si atoms, list in descending order, and change the second one from the top
si_atoms_coordinates = [coordinate for coordinate in slab.basis.coordinates.values]
si_atoms_sorted = sorted(si_atoms_coordinates, key=lambda x: x[2], reverse=True)
second_from_top_index = 1
second_si_atom_coordinate = si_atoms_sorted[second_from_top_index]

print(f"Coordinates of the second Si atom: {second_si_atom_coordinate}")
adjusted_coordinate = [
    coord + delta for coord, delta in zip(second_si_atom_coordinate, [0.025, 0, 0.025])
]
print(f"Adjusted coordinate: {adjusted_coordinate}")
new_coordinates = si_atoms_coordinates.copy()
index_to_adjust = slab.basis.coordinates.get_element_id_by_value(second_si_atom_coordinate)
new_coordinates[index_to_adjust] = adjusted_coordinate
slab.set_coordinates(new_coordinates)

### 2.3. Visualize adjusted material

In [ ]:
if SHOW_INTERMEDIATE_STEPS:
    print("Adjusted slab structure:")
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 3. Create passivated material
### 3.1. Set up passivation configuration and builder


In [ ]:
from mat3ra.made.tools.build.passivation import (
    PassivationConfiguration,
    SurfacePassivationBuilder,
    SurfacePassivationBuilderParameters
)

config = PassivationConfiguration(
    slab=slab,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH,
    surface=SURFACE
)

builder_params = SurfacePassivationBuilderParameters(
    shadowing_radius=SHADOWING_RADIUS,
    depth=DEPTH
)

builder = SurfacePassivationBuilder(build_parameters=builder_params)

### 3.2. Generate passivated structure


In [ ]:
from mat3ra.made.tools.build.passivation import create_passivation

passivated_slab = create_passivation(config, builder)

print("\nPassivation completed:")
print(f"Passivant used: {PASSIVANT}")
print(f"Bond length: {BOND_LENGTH} Å")
print(f"Surfaces passivated: {SURFACE}")
print(f"Number of atoms: {len(passivated_slab.basis.elements.ids)}")

## 4. Visualize the result

In [ ]:
print("Final passivated structure:")
visualize_materials(passivated_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(passivated_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(passivated_slab)
download_content_to_file(passivated_slab, f"{slab.name}_passivated.json")